# Generar piezas musicales con el dataset de Spanio 🎵

1. Usar los prompts de Spanio y pasarlos a csv https://github.com/matteospanio/taste-music-dataset/blob/main/descriptions.json

2. Cargar el modelo tasty-musicgen-small https://github.com/matteospanio/tasty-musicgen-small

3. Definir e implementar un pipeline para generar piezas musicales a partir de los prompts.

4. Calcular el CLAP Score de las piezas musicales.

5. Crear un script que ejecute el pipeline de generación de audio.

In [ ]:
import yaml
import csv
import os
import pandas as pd
import json

import scipy.io.wavfile
import torch
import torchaudio

from laion_clap import CLAP_Module
from torch.utils.data import Dataset
from tqdm import tqdm

from transformers import pipeline
from tqdm import tqdm

/home/juana/.venv311/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Cargar los prompts de Spanio

In [20]:
def load_config(path="config.yaml"):
    """
    Carga un archivo de configuración en formato YAML.

    Parameters:
        path (str): Ruta al archivo YAML de configuración. Por defecto es "config.yaml".

    Returns:
        dict: Diccionario con la configuración cargada desde el archivo YAML.
    """
    with open(path, "r") as f:
        return yaml.safe_load(f)

config = load_config()

In [21]:
data_docs_path = config["data_docs_path"]
data_prompts_path = config["data_prompts_path"]
model_musicgen_path = config["model_musicgen_path"]
tracks_data_path = config["tracks_data_path"]
data_clap_path = config["data_clap_path"]

In [4]:
# 1. Base de documentos (JSON normal).
with open(data_docs_path, 'r') as f:
    docs_data = json.load(f)

df_docs = pd.DataFrame(docs_data)
print("Documentos cargados:", df_docs.shape)

Documentos cargados: (100, 3)


In [5]:
df_docs

,id,instrument,description
0,1,piano,A sweet melancholic piano piece.
1,2,piano and strings,A bitter-sweet dreamy piano piece.
2,3,piano,A sweet-salty piece with a continuous arpeggio...
3,4,plucked strings,"A bitter piece, a little salty. This is mainly..."
4,5,piano,A sweet-bitter static piano piece. Really slow...
...,...,...,...
95,96,marimba and drums,A sweet and salty piece. The marimba plays a r...
96,97,guitar,A sweet and salty piece. The guitar plays a ry...
97,98,electronic,A sweet and salty piece. The bass plays a ryth...
98,99,"brass, bass and percussions",A salty and sweet piece. The brass play the ac...


In [ ]:
class LoadSpanioDataset(Dataset):
    """
    Clase `Dataset` para cargar, transformar y exportar descripciones musicales
    del conjunto de datos de Spanio (`taste-music-dataset`).

    Esta clase permite leer un archivo JSON con estructura de columnas o lista
    de registros, convertirlo a una lista de diccionarios individuales,
    acceder a sus elementos por índice y exportarlos a CSV.

    Attributes:
        json_file_path (str): La ruta al archivo JSON que contiene los datos de Spanio.
    """

    def __init__(self, json_file_path):
        """
        Inicializa la clase `LoadSpanioDataset` cargando el contenido del archivo JSON.

        Parameters:
            son_file_path (str): La ruta al archivo JSON que contiene los datos de Spanio.
        """
        super().__init__()
        self.json_file_path = json_file_path
        self.records = []
        self._load_data()

    def _load_data(self):
        """
        Carga los datos del archivo JSON y normaliza su estructura.
        """
        try:
            with open(self.json_file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                # Normalizar estructura (lista o dict).
                self.records = (
                    data if isinstance(data, list)
                    else [{"id": k, **v} for k, v in data.items()]
                )
            print(f"Cargados {len(self.records)} registros desde {self.json_file_path}")
        except Exception as e:
            print(f"Error cargando {self.json_file_path}: {e}")

    def __len__(self):
        """
        Retorna el número total de registros (extractos) en el dataset.
        
        Returns:
            int: Número de registros disponibles.
        """
        return len(self.records)

    def __getitem__(self, idx):
        """
        Retorna un registro específico del dataset por índice.

        Parameters:
            idx (int): Índice del registro a retornar.

        Returns
            dict: Diccionario con las llaves `id`, `instrument` y `description`.
        """
        if not 0 <= idx < len(self.records):
            raise IndexError(f"indice {idx} fuera de rango para dataset: {len(self.records)}.")
        return self.records[idx]
    
    
    def map_records(self):
        """
        Mapea los registros de self.records() a un diccionario.

        Cada clave del diccionario es el 'id' del registro, y su valor es otro
        diccionario con el 'content' y 'title' del registro.
        
        Returns
        
        dict:
            - Diccionario donde las llaves son los `id` de los registros y los valores.
            - son diccionarios con `instrument` y `description`.

        """
        return {
            doc["id"]: {
                "instrument": doc["instrument"],
                "description": doc["description"]
            }
            for doc in self.records
        }
        
    def to_csv(self, output_path="spanio_prompts.csv"):
        """
        Exporta los registros del dataset a un archivo CSV con columnas:
        `id`, `instrument`, `description`.
        
        Parameters
            output_path (str): Ruta de salida donde se guardará el archivo CSV.

        """
        try:
            with open(output_path, "w", newline='', encoding="utf-8") as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=["id", "instrument", "description"])
                writer.writeheader()
                writer.writerows(self.records)
            print(f"CSV generado en: {output_path}")
        except Exception as e:
            print(f"Error exportando a CSV: {e}")

In [7]:
dataset_spanio = LoadSpanioDataset(data_docs_path)

Cargados 100 registros desde ../data/descriptions.json


In [8]:
dataset_spanio.to_csv(data_prompts_path)

CSV generado en: ../data/spanio_prompts.csv


In [9]:
print("\n🔹 Ejemplo de registro:")
print(dataset_spanio[0])


🔹 Ejemplo de registro:
{'id': 1, 'instrument': 'piano', 'description': 'A sweet melancholic piano piece.'}


### 2. Cargar el modelo tasty-musicgen-small

El pipeline crea un síntetizador de audio que usa el tasty-musicgen-small para la tarea "text-to-audio".

| Parámetro                   | Propósito                                    | 
| --------------------------- | -------------------------------------------- | 
| `"text-to-audio"`           | Define la tarea (texto → audio). Hugging Face tiene varios tipos de tareas: "text-generation", "image-classification", "automatic-speech-recognition"              | 
| `model=model_musicgen_path` | Ruta o nombre del modelo MusicGen.            | 
| `device=-1`                 | Ejecuta en CPU. Cambia a `device=0` para GPU. |                    |
| `trust_remote_code=True`    | Permite usar código personalizado del modelo. Muchos modelos modernos (como tasty-musicgen-small) incluyen su propia implementación de clases o funciones personalizadas dentro de su repositorio. Si no se activa este parámetro, el pipeline usaría solo las clases predefinidas de transformers, y fallaría si el modelo depende de código remoto. | Necesario para modelos “custom” de Hugging Face |


In [11]:
synthesiser = pipeline(
    "text-to-audio",
    model=model_musicgen_path,
    device=-1,  # CPU
    trust_remote_code=True
)


Device set to use cpu


In [12]:
print(synthesiser.model.__class__)
print(synthesiser.model.device)

<class 'transformers.models.musicgen.modeling_musicgen.MusicgenForConditionalGeneration'>
cpu


### 3. Definir e implementar un pipeline para generar piezas musicales a partir de los prompts

¿Qué es el `sample_rate`?

- El sample_rate (frecuencia de muestreo) indica cuántas muestras de audio se registran por segundo.

- `sample_rate = 32000` equivale a 32.000 muestras por segundo.

- Cuantas más muestras por segundo, mayor fidelidad y detalle tiene el sonido, pero también aumenta el tamaño del archivo y el costo computacional.

"tasty-musicgen-small is a musicgen-small fine-tuned on a patched version of the Taste & Affect Music Database. It generates music that's supposed to induce gustatory synesthesia perceptions based on multimodal research. It generates mono audio in **32khz**" ([referencia](https://huggingface.co/csc-unipd/tasty-musicgen-small)).

`synthesiser(text_prompt, forward_params={"do_sample": True})`

Invoca internamente:

- El processor/tokenizer que convierte el texto en tensores.

- El método model.generate(...) del modelo MusicGen con parámetros por defecto + forward_params.

- El post-procesado que decodifica los códigos o embeddings a una señal de audio (PCM float).

Nota: PCM es el proceso de convertir una señal de audio analógica (onda continua) en una señal digital (datos discretos) 

Parámetro `forward_params={"do_sample": True})`

Es un diccionario que pasa argumentos a model.generate() o al método interno que hace la inferencia.

- `{"do_sample": True}` habilita muestreo estocástico en la generación (aleatoriedad) en vez de usar greedy decoding; produce variaciones creativas entre ejecuciones.

In [31]:
def generate_music_from_prompts(synthesiser, dataset, output_dir="generated_music", sample_rate=32000):
    """
    Genera archivos de audio a partir de descripciones de texto usando el modelo tasty-musicgen-small.
    
    Parameters:
        synthesiser: Pipeline de Hugging Face para text-to-audio.
        dataset: Instancia de LoadSpanioDataset con prompts.
        output_dir (str): Carpeta donde guardar los .wav generados.
        sample_rate (int): Frecuencia de muestreo para los archivos de salida. MusicGen fue entrenado a 32 kHz.
        
    Returns:
        list[dict]: Lista con {'id', 'instrument', 'description', 'audio_path'} por cada generación.
    """
    os.makedirs(output_dir, exist_ok=True)
    results = []

    print(f"Generando música para {len(dataset)} prompts...\n")

    for record in tqdm(dataset.records):
        text_prompt = record["description"]
        file_id = record["id"]

        try:
            # 1. Generar la música con el modelo.
            # output es un diccionario: audio(array NumPy con la señal de audio) y sampling_rate (frecuencia de muestreo del modelo).
            output = synthesiser(text_prompt, forward_params={"do_sample": True})
            
            # 2. Extraer datos del audio.
            audio_data = output["audio"] # La onda de sonido (las muestras del audio).
            sr = output.get("sampling_rate", sample_rate) # La frecuencia de muestreo reportada por el modelo.

            # 3. Guardar el audio generado.
            output_path = os.path.join(output_dir, f"{file_id}.wav")
            scipy.io.wavfile.write(output_path, rate=sr, data=audio_data) # Escribir el archivo .wav con la señal y la frecuencia.

            # 4. Registrar los resultados.
            results.append({
                "id": file_id,
                "instrument": record["instrument"],
                "description": text_prompt,
                "audio_path": output_path
            })
        except Exception as e:
            print(f" Error generando {file_id}: {e}")
            continue

    print(f"\n {len(results)} archivos de audio generados en: {output_dir}")
    return results

In [ ]:
results = generate_music_from_prompts(synthesiser, dataset_spanio, tracks_data_path)

Ejemplo para el primer registro:

In [13]:
record = dataset_spanio[0]
record

{'id': 1,
 'instrument': 'piano',
 'description': 'A sweet melancholic piano piece.'}

In [29]:
sample_rate=32000
results = []
os.makedirs(tracks_data_path, exist_ok=True)

In [ ]:
text_prompt = record["description"]
file_id = record["id"]

try:
    # Generar música.
    output = synthesiser(text_prompt, forward_params={"do_sample": True})
    audio_data = output["audio"]
    sr = output.get("sampling_rate", sample_rate)

    # Guardar archivo.
    output_path = os.path.join(tracks_data_path, f"{file_id}.wav")
    scipy.io.wavfile.write(output_path, rate=sr, data=audio_data)

    results.append({
        "id": file_id,
        "instrument": record["instrument"],
        "description": text_prompt,
        "audio_path": output_path
    })
except Exception as e:
    print(f" Error generando {file_id}: {e}")

In [32]:
results

[{'id': 1,
  'instrument': 'piano',
  'description': 'A sweet melancholic piano piece.',
  'audio_path': '../data/generated_music/1.wav'}]

### Paso 4: Calcular el CLAP Score

`CLAP_Module`:

Es un wrapper del modelo CLAP que permite el uso del modelo CLAP para obtener embeddings de audio y texto, calcular similitudes o entrenar nuevos modelos multimodales.

Parámetro:

- `enable_fusion`: activa o desactiva un mecanismo interno del modelo CLAP que combina información de audio y texto en una representación conjunta, es decir, un embedding fusionado. Esto permite calcular de forma directa una similitud entre embeddings audio ↔ texto, sin necesidad de entrenar un modelo adicional. Si es False, el modelo cargaría solo el codificador de audio o texto, sin capacidad de comparar entre ellos, por lo que el cálculo de similitud coseno no tendría sentido.

Fórmula de la similitud del coseno:

$$
\text{sim}(a, b) = \frac{a \cdot b}{\|a\| \|b\|}
$$

El resultado (score) es un número entre -1 y 1:

- +1 → audio y texto son muy similares.

- 0 → no hay relación.

- -1 → son opuestos semánticamente (raro en práctica).

¿Por qué en CLAP casi nunca salen negativos?

El modelo CLAP fue entrenado con una pérdida contrastiva tipo InfoNCE que:

- Maximiza la similitud entre los pares correctos (audio ↔ descripción) 
- Minimiza la similitud entre los pares incorrectos.

El modelo nunca vio ejemplos de “oposición semántica” (como “silencio” vs “explosión”) durante el entrenamiento, por eso, el coseno rara vez llega a valores extremos (−1 o 1).

Por lo tanto, tras el entrenamiento:

| Tipo de relación audio-texto | CLAP Score típico |
| ---------------------------- | ----------------- |
| Muy alta coherencia          | 0.7 – 0.9         |
| Moderada coherencia          | 0.4 – 0.6         |
| Poca coherencia              | 0.2 – 0.4         |
| Ruido o sin relación         | < 0.2             |

[Ver referencia de clap score](https://arxiv.org/html/2506.23553v2)

In [ ]:
def compute_clap_scores(results, device=None):
    """
    Calcula el CLAP Score (similaridad texto-audio) usando embeddings del modelo CLAP.

    Parameters:
        results (list[dict]): Lista de diccionarios con llaves 'audio_path' y 'description'.
        device (str, opcional): Dispositivo ('cuda' o 'cpu'). Si None, detecta automáticamente.

    Returns:
        list[dict]: Misma lista de entrada, agregando la llave 'clap_score' (float).
    """
    # 1. Configurar dispositivo.
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\nUsando dispositivo: {device}\n")

    # 2. Cargar modelo CLAP.
    clap_model = CLAP_Module(enable_fusion=True) # Activa la modalidad combinada audio-texto del modelo.
    clap_model.load_ckpt() # Descarga y carga los pesos preentrenados.
    clap_model.eval() # Modo evaluación (desactiva dropout, gradientes, etc.).
    clap_model.to(device)

    print("Modelo CLAP cargado correctamente.\nCalculando CLAP Scores...\n")

    scored = []

    # 3. Iterar sobre los resultados.
    for r in tqdm(results, desc="Procesando audios", ncols=80):
        # 4. Carga y preprocesamiento del audio.
        try:
            audio, sr = torchaudio.load(r["audio_path"]) # Carga el audio en un tensor y su frecuencia de muestreo.
            if sr != 48000:
                audio = torchaudio.functional.resample(audio, sr, 48000) # Si no está a 48 kHz, lo resamplea.
            audio = audio.to(device) # Enviar a dispositivo.

            with torch.no_grad():
                # 5. Obtener embeddings.
                audio_emb = clap_model.get_audio_embedding_from_data(audio, use_tensor=True)
                text_emb = clap_model.get_text_embedding([r["description"]], use_tensor=True)
                
                # 6. Normalización y cálculo de similitud.
                audio_emb = torch.nn.functional.normalize(audio_emb, dim=-1)
                text_emb = torch.nn.functional.normalize(text_emb, dim=-1)

                # Calcular similitud coseno.
                score = torch.nn.functional.cosine_similarity(audio_emb, text_emb).item()

            # 7. Guardar el resultado.
            r["clap_score"] = round(float(score), 6)
            scored.append(r)

        except Exception as e:
            print(f"Error calculando CLAP para {r.get('id', '?')} ({r['audio_path']}): {e}")
    return scored


In [ ]:
scored_results = compute_clap_scores(results)


Usando dispositivo: cpu



'(ProtocolError('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer')), '(Request ID: b53d4055-ae67-4cc2-b637-d503a900c0b0)')' thrown while requesting HEAD https://huggingface.co/roberta-base/resolve/main/config.json
Retrying in 1s [Retry 1/5].
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

Procesando audios:   0%|                                  | 0/1 [00:00<?, ?it/s]/home/juana/.venv311/lib64/python3.11/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/home/juana/.venv311/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:581: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(
Procesando audios: 100%|██████████████████████████| 1/1 [00:02<00:00,  2.85s/it]


In [ ]:
scored_results

[{'id': 1,
  'instrument': 'piano',
  'description': 'A sweet melancholic piano piece.',
  'audio_path': '../data/generated_music/1.wav',
  'clap_score': 0.408918}]

### Paso 5: Script completo para ejecutar el pipeline

Ejecutar el pipeline para los primeros tres registros:

In [33]:
dataset = LoadSpanioDataset(data_docs_path)
dataset.records = dataset.records[:3]

synthesiser = pipeline(
    "text-to-audio",
    model=model_musicgen_path,
    device=-1,
    trust_remote_code=True
)

results = generate_music_from_prompts(synthesiser, dataset, tracks_data_path)
scored_results = compute_clap_scores(results)

Cargados 100 registros desde ../data/descriptions.json


Device set to use cpu


Generando música para 3 prompts...



100%|██████████| 3/3 [45:09<00:00, 903.02s/it]



 3 archivos de audio generados en: ../data/generated_music

Usando dispositivo: cpu



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

Procesando audios:   0%|                                  | 0/3 [00:00<?, ?it/s]/home/juana/.venv311/lib64/python3.11/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/home/juana/.venv311/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:581: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(
Procesando audios: 100%|██████████████████████████| 3/3 [00:04<00:00,  1.57s/it]


In [34]:
df_clap = pd.DataFrame(scored_results)
df_clap.to_csv(data_clap_path, index=False)
print("\nPipeline completo: resultados guardados en csv")


Pipeline completo: resultados guardados en csv


In [36]:
df_clap

,id,instrument,description,audio_path,clap_score
0,1,piano,A sweet melancholic piano piece.,../data/generated_music/1.wav,0.411625
1,2,piano and strings,A bitter-sweet dreamy piano piece.,../data/generated_music/2.wav,0.397569
2,3,piano,A sweet-salty piece with a continuous arpeggio...,../data/generated_music/3.wav,0.120244


In [ ]:
if __name__ == "__main__":
    dataset = LoadSpanioDataset(data_docs_path)
    
    synthesiser = pipeline(
        "text-to-audio",
        model=model_musicgen_path,
        device=-1,
        trust_remote_code=True
    )

    results = generate_music_from_prompts(synthesiser, dataset)
    scored_results = compute_clap_scores(results)

    df = pd.DataFrame(scored_results)
    df.to_csv(data_clap_path, index=False)
    print("\nPipeline completo: resultados guardados en csv")